In [1]:
from analyze import *

In [2]:
evaluator = TestEvaluator('../../data/raw/openja/lightfm_demo')
evaluator.load_checklist('../../checklist/checklist_demo.yaml')

evaluator.evaluate()

  0%|                                                                                                                                                                                                                   | 0/3 [00:00<?, ?it/s]

../../data/raw/openja/lightfm_demo/tests/test_cross_validation.py
# splits: 3


 33%|███████████████████████████████████████████████████████████████████▋                                                                                                                                       | 1/3 [00:09<00:18,  9.18s/it]

../../data/raw/openja/lightfm_demo/tests/test_evaluation.py
# splits: 3


 67%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                   | 2/3 [00:18<00:09,  9.30s/it]

../../data/raw/openja/lightfm_demo/tests/test_data.py
# splits: 3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.25s/it]


In [3]:
score = evaluator.get_completeness_score()

Report:
                                                   is_Satisfied  \
ID  Title                                                         
1.1 Write Descriptive Test Names                            1.0   
1.2 Keep Tests Focused                                      1.0   
2.1 Ensure Data File Loads as Expected                      0.5   
5.1 Validate Model Input and Output Compatibility           1.0   

                                                   n_files_tested  \
ID  Title                                                           
1.1 Write Descriptive Test Names                                3   
1.2 Keep Tests Focused                                          3   
2.1 Ensure Data File Loads as Expected                          3   
5.1 Validate Model Input and Output Compatibility               3   

                                                                                           functions  
ID  Title                                                              

In [4]:
reports = pd.DataFrame(evaluator.evaluation_result)['report'].explode('report').apply(pd.Series)
reports['file'] = reports['file'].str[35:]
reports.query('ID == "5.1"')[['ID', 'Title', 'file', 'Evaluation', 'Score']]#.to_dict('records')

,ID,Title,file,Evaluation,Score
3,5.1,Validate Model Input and Output Compatibility,tests/test_cross_validation.py,Satisfied,1.0
7,5.1,Validate Model Input and Output Compatibility,tests/test_evaluation.py,Satisfied,1.0
11,5.1,Validate Model Input and Output Compatibility,tests/test_data.py,Satisfied,1.0


In [5]:
reports.query('ID == "2.1"')[['ID', 'Title', 'file', 'Evaluation', 'Score']]

,ID,Title,file,Evaluation,Score
2,2.1,Ensure Data File Loads as Expected,tests/test_cross_validation.py,Partially Satisfied,0.5
6,2.1,Ensure Data File Loads as Expected,tests/test_evaluation.py,Not Satisfied,0.0
10,2.1,Ensure Data File Loads as Expected,tests/test_data.py,Not Satisfied,0.0


In [6]:
reports

,ID,Title,Requirement,Observation,Functions,Evaluation,Score,file
0,1.1,Write Descriptive Test Names,"Each test function should have a clear, descri...",The test function 'test_random_train_test_spli...,[test_random_train_test_split],Satisfied,1.0,tests/test_cross_validation.py
1,1.2,Keep Tests Focused,"Each test should focus on a single scenario, u...",The test function 'test_random_train_test_spli...,[test_random_train_test_split],Satisfied,1.0,tests/test_cross_validation.py
2,2.1,Ensure Data File Loads as Expected,Ensure that data-loading functions correctly l...,The test function 'test_random_train_test_spli...,[test_random_train_test_split],Partially Satisfied,0.5,tests/test_cross_validation.py
3,5.1,Validate Model Input and Output Compatibility,Confirm that the model accepts inputs of the c...,The test function 'test_random_train_test_spli...,[test_random_train_test_split],Satisfied,1.0,tests/test_cross_validation.py
4,1.1,Write Descriptive Test Names,"Each test function should have a clear, descri...",The test functions have clear and descriptive ...,"[test_precision_at_k, test_precision_at_k_with...",Satisfied,1.0,tests/test_evaluation.py
5,1.2,Keep Tests Focused,"Each test should focus on a single scenario, u...","Each test focuses on a single scenario, using ...","[test_precision_at_k, test_precision_at_k_with...",Satisfied,1.0,tests/test_evaluation.py
6,2.1,Ensure Data File Loads as Expected,Ensure that data-loading functions correctly l...,The provided test functions do not involve loa...,[],Not Satisfied,0.0,tests/test_evaluation.py
7,5.1,Validate Model Input and Output Compatibility,Confirm that the model accepts inputs of the c...,The test functions validate the model's input ...,"[test_precision_at_k, test_precision_at_k_with...",Satisfied,1.0,tests/test_evaluation.py
8,1.1,Write Descriptive Test Names,"Each test function should have a clear, descri...",The test functions have clear and descriptive ...,"[test_fitting, test_fitting_no_identity, test_...",Satisfied,1.0,tests/test_data.py
9,1.2,Keep Tests Focused,"Each test should focus on a single scenario, u...",The test functions focus on specific scenarios...,"[test_fitting, test_fitting_no_identity, test_...",Satisfied,1.0,tests/test_data.py
